In [1]:
import dask.dataframe as dd
from datetime import datetime
import math
from dask.multiprocessing import get
from dask.distributed import Client, LocalCluster 
print('test')
import dask

#cluster = LocalCluster(n_workers = 4, threads_per_worker = 4, processes=False, memory_limit='4GB', silence_logs='error')
client = Client(processes=False, silence_logs='error')
#client = Client(cluster)
#cluster
client

test


Client Scheduler: inproc://192.168.132.2/99915/1 Dashboard: http://192.168.132.2/99915/1:8787/status,Cluster Workers: 1 Cores: 12 Memory: 17.18 GB


In [2]:
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [3]:
# fillna - used because null/None is float, not str, had to make it a str, for max/min
def get_inventory(file_type:str) -> dd:
    if file_type == 'parquet':
        df = dd.read_parquet('./data/checkouts/inventory_parquet')
    elif 'csv' in file_type:
        df = (dd
          .read_csv("./data/checkouts/Library_Collection_Inventory.csv",
                    header=0, 
                    dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str"})
         )  
    df = df.drop_duplicates(subset='BibNum')
    return df
    

def get_checkouts(file_type:str) -> dd:
    if file_type == 'multi-csv':
        df = dd.read_csv("./data/checkouts/Checkouts_By_Title_Data_Lens_*.csv", 
                         header=0, 
                         dtype={"BibNumber":'int64',
                                "ItemBarcode":"str",
                                "ItemType":"str",
                                "Collection":"str",
                                "CallNumber":"str",
                                "CheckoutDateTime":"str"}
                        )
    elif file_type =='single-csv':
        df = dd.read_csv("./data/checkouts/Checkouts.csv", 
                         header=0, 
                         dtype={"BibNumber":'int64',
                                "ItemBarcode":"str",
                                "ItemType":"str",
                                "Collection":"str",
                                "CallNumber":"str",
                                "CheckoutDateTime":"str"}
                        )
    elif file_type =='parquet':
        df = dd.read_parquet('./data/checkouts/checkouts_parquet')  
    return df

def writter_csv(df: dd, scheduler_type, single_file: bool) -> None:
    if single_file:
        df.to_csv('./output/output.csv',single_file=single_file, compute_kwargs={"scheduler": scheduler_type}, mode="w+")
    else:
        df.to_csv('./output/output',single_file=single_file, compute_kwargs={"scheduler": scheduler_type}, mode="w+")
        
def writter_parquet(df: dd, scheduler, flag=None) ->None:
    df.to_parquet("./output/parquet_output")


In [4]:
computes = ["threads", "processes","synchronous"]
file_types = ["parquet", "multi-csv", "single-csv"]
wr = [[writter_csv, False], [writter_parquet,False],[writter_csv, True]]

In [12]:
%timeit -n 1 -r 5 get_checkouts("parquet")
%timeit -n 1 -r 5 get_checkouts("multi-csv")
%timeit -n 1 -r 5 get_checkouts("single-csv")

2.38 s ± 269 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.06 s ± 452 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
534 ms ± 260 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [7]:
%timeit -n 1 -r 5 get_checkouts("parquet").head()
%timeit -n 1 -r 5 get_checkouts("multi-csv").head()
%timeit -n 1 -r 5 get_checkouts("single-csv").head()

1.02 s ± 12.9 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.58 s ± 26.3 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.51 s ± 11.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [16]:
for compute in computes:
    for file_type in file_types:
        print(f"Checkouts. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 5 get_checkouts("parquet").count().compute()


26.7 ms ± 7.29 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
78.1 ms ± 12.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
76.2 ms ± 1.04 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
for compute in computes:
    for file_type in file_types:
        print(f"Checkouts. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 5 get_checkouts("parquet").compute()


In [13]:
#Read and write
def scenario1(writter, single_file, compute, file_type):
    df = get_checkouts(file_type)
    writter(df, compute, single_file)
#Read and write
def scenario1_5(writter, single_file, compute, file_type):
    df = get_inventory(file_type)
    writter(df, compute, single_file)

In [14]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"Checkouts. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario1(writter[0], writter[1], compute, file_type)
            print(f"Inventory. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario1_5(writter[0], writter[1], compute, file_type)

Checkouts. compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed |  8min 34.9s
8min 34s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Inventory. compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 16.9s
16.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Checkouts. compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
5min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Inventory. compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
12.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Checkouts. compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed |  9min 52.7s
9min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

12.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Checkouts. compute: synchronous, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed |  8min 25.2s
8min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Inventory. compute: synchronous, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 19.8s
19.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Checkouts. compute: synchronous, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed |  9min 28.5s
9min 28s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Inventory. compute: synchronous, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 21.1s
21.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Checkouts.

In [15]:
def add_formatted_checkout(file_type:str)->dd:
    df = get_checkouts(file_type)
    df['CheckoutTime_formated']=dd.to_datetime(df.CheckoutDateTime, format="%m/%d/%Y %I:%M:%S %p")
    return df

In [16]:
#read add a column and write
def scenario2(writter, single_flag, scheduler, file_type):
    df = add_formatted_checkout(file_type)
    writter(df,scheduler, single_flag)

In [17]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario2(writter[0], writter[1], compute, file_type)

compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 15min 26.1s
15min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
7min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 17min 44.6s
17min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 16min  9.8s
16min 9s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_parquet, single_file = False
8min 24s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-

In [61]:
#read, add a column and agg
def scenario3(scheduler,  file_type):
    df = add_formatted_checkout(file_type)
    df['month'] = df['CheckoutTime_formated'].dt.to_period('M')
    df = df.month.value_counts()
    df = dask.optimize(df)[0]
    df.compute(scheduler=scheduler)
    

In [62]:
for compute in computes:
    for file_type in file_types:
        print(f"compute: {compute}, file_type: {file_type}, writter: NO, single_file = NO")
        %timeit -n 1 -r 1 scenario3(compute, file_type)
            

compute: threads, file_type: parquet, writter: NO, single_file = NO
[########################################] | 100% Completed |  6min 18.7s
6min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: NO, single_file = NO
[########################################] | 100% Completed |  6min 44.5s
6min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: single-csv, writter: NO, single_file = NO
[########################################] | 100% Completed |  6min 47.5s
6min 47s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: parquet, writter: NO, single_file = NO
[########################################] | 100% Completed |  1min 37.9s
1min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: multi-csv, writter: NO, single_file = NO
[########################################] | 100% Completed |  1min 42.6s
1min 46s ± 0 ns

In [63]:
# read and get row count Scenario3
for compute in computes:
    for file_type in file_types:
        print(f"compute: {compute}, file_type: {file_type}")
        %timeit -n 1 -r 1 get_checkouts(file_type).index.count().compute(scheduler=compute)

compute: threads, file_type: parquet
[########################################] | 100% Completed |  1min 43.9s
1min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv
[########################################] | 100% Completed |  2min  4.9s
2min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: single-csv
[########################################] | 100% Completed |  2min  8.0s
2min 8s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: parquet
[########################################] | 100% Completed | 41.7s
45.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: multi-csv
[########################################] | 100% Completed | 37.7s
41.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: single-csv
[########################################] | 100% Completed | 40.0s
44.1 s ± 0 n

In [50]:
%%timeit -n 1 -r 1 
#read, column transform + min value of new column compute
for compute in computes:
    for file_type in file_types:
        print(f"compute: {compute}, file_type: {file_type}")
        df = add_formatted_checkout(file_type)
        df = df.CheckoutTime_formated.min()
        df = dask.optimize(df)[0]
        print(df.compute(scheduler=compute))

compute: threads, file_type: parquet
[########################################] | 100% Completed |  6min  9.0s
2005-04-13 08:00:00
compute: threads, file_type: multi-csv
[########################################] | 100% Completed |  6min 35.8s
2005-04-13 08:00:00
compute: threads, file_type: single-csv
[########################################] | 100% Completed |  6min 44.6s
2005-04-13 08:00:00
compute: processes, file_type: parquet
[########################################] | 100% Completed |  1min 39.4s
2005-04-13 08:00:00
compute: processes, file_type: multi-csv
[########################################] | 100% Completed |  1min 49.0s
2005-04-13 08:00:00
compute: processes, file_type: single-csv
[########################################] | 100% Completed |  1min 46.7s
2005-04-13 08:00:00
compute: synchronous, file_type: parquet
[########################################] | 100% Completed |  6min  8.1s
2005-04-13 08:00:00
compute: synchronous, file_type: multi-csv
[###################

In [93]:
#read, join, write, one Index
def scenario4_1(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber","ItemBarcode"]]
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    inventory = inventory.set_index("BibNum")
    merged = dd.merge(checkouts, inventory, left_on='BibNumber', right_index=True, how="inner") 
    writter(merged, scheduler_type, single_flag)

In [94]:
def scenario4(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber","ItemBarcode"]]
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    merged = dd.merge(checkouts, inventory, left_on='BibNumber', right_on="BibNum", how="inner") 
    writter(merged, scheduler_type, single_flag)

In [95]:
#read, join, write, Two Indexes
def scenario4_2(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber","ItemBarcode"]]
    checkouts = checkouts.set_index("BibNumber")
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    inventory = inventory.set_index("BibNum")
    merged = dd.merge(checkouts, inventory, left_index=True, right_index=True, how="inner")
    writter(merged, scheduler_type, single_flag)

In [118]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"No Index compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario4(writter[0], writter[1], compute, file_type)
            """
            print(f"One Index compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario4_1(writter[0], writter[1], compute, file_type)
            #Error on dask.join /dask.merge on both indexes with parquet files!
            if not (file_type=="parquet" and compute=="threads" and {writter[0].__name__}=="writter_parquet"):
                print(f"two index compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
                %timeit -n 1 -r 1 scenario4_2(writter[0], writter[1], compute, file_type)
            """

No Index compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed |  7min 36.5s
7min 36s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
No Index compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
4min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
No Index compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed |  7min 16.9s
7min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
No Index compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed |  7min 38.6s
7min 38s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
No Index compute: threads, file_type: multi-csv, writter: writter_parquet, single_file = False
4min 24s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loo

In [77]:
#read, join, group by + count , compute top 10
def scenario5(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber", "ItemBarcode"]]
    checkouts = checkouts.set_index("BibNumber")
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    inventory = inventory.set_index("BibNum")
    merged = dd.merge(checkouts, inventory, left_index=True, right_index=True, how="inner") 
    merged = merged.Author.value_counts().nlargest(10)
    merged = dask.optimize(merged)[0]
    merged.compute()
   

In [78]:
#read, join, group by + count , compute top 10
def scenario5_1(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber", "ItemBarcode"]]
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    inventory = inventory.set_index("BibNum")
    merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_index=True, how="inner") 
    merged = merged.Author.value_counts().nlargest(10)
    merged = dask.optimize(merged)[0]
    merged.compute()
   

In [80]:
#read, join, group by + count , compute top 10
def scenario5_2(writter, single_flag, scheduler_type, file_type):
    checkouts = get_checkouts(file_type)[["BibNumber", "ItemBarcode"]]
    inventory = get_inventory(file_type)[["BibNum", "Author"]]
    merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner") 
    merged = merged.Author.value_counts().nlargest(10)
    merged = dask.optimize(merged)[0]
    merged.compute()
   

In [81]:
for compute in computes:
    for file_type in file_types:
        print(f"no index. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 1 scenario5_2(writter[0], writter[1], compute, file_type)
        """
        print(f"inventory index. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 1 scenario5_1(writter[0], writter[1], compute, file_type)
        print(f"two index. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 1 scenario5(writter[0], writter[1], compute, file_type)
        """
        
       
    

no index. compute: threads, file_type: parquet, writter: writter_csv, single_file = False
2min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
inventory index. compute: threads, file_type: parquet, writter: writter_csv, single_file = False
3min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
two index. compute: threads, file_type: parquet, writter: writter_csv, single_file = False
14min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
no index. compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
3min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
inventory index. compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
3min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
two index. compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
15min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
no index. compute: threads, file_type: single-

In [82]:
#read, string split, explode, write
def scenario7(writter, single_flag, scheduler_type, file_type):
    inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
    inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
    inventory = inventory.explode("Subject")
    inventory = dask.optimize(inventory)[0]
    writter(inventory, scheduler_type, single_flag)


In [83]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario7(writter[0], writter[1], compute, file_type)
            

compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 26.9s
27 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
17.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 26.0s
26 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 29.1s
29.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_parquet, single_file = False
18.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_

In [84]:
#read, string split, explode, agg + count distinct, write
def scenario8(writter, single_flag, scheduler_type, file_type):
    inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
    inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
    inventory = inventory.explode("Subject")
    inventory = inventory.set_index("Subject")
    inventory = inventory.map_partitions(lambda x: x.groupby(x.index)["BibNum"].nunique().astype("int64"))
    inventory = dask.optimize(inventory)[0]
    inventory.compute(scheduler=scheduler_type)
    
  

In [85]:
for compute in computes:
    for file_type in file_types:
        print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
        %timeit -n 1 -r 1 scenario8(writter[0], writter[1], compute, file_type)
            

compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 23.1s
42.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 24.5s
45.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: single-csv, writter: writter_csv, single_file = True
[########################################] | 100% Completed | 25.1s
46.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed |  4min 39.1s
5min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: processes, file_type: multi-csv, writter: writter_csv, single_file = True
[########################################] | 100% Completed

In [13]:
def scenario9(writter, single_flag, scheduler_type, file_type):
    inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
    inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
    inventory = inventory.explode("Subject")
    inventory = inventory.set_index("Subject")
    inventory = inventory.groupby(inventory.index)["BibNum"].nunique().astype("int64").reset_index()
    inventory = dask.optimize(inventory)[0]
    writter(inventory,scheduler_type, single_flag)
  

In [14]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 1 scenario9(writter[0], writter[1], compute, file_type)


compute: threads, file_type: parquet, writter: writter_csv, single_file = False
[########################################] | 100% Completed | 11min 21.1s
12min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
8min 45s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: parquet, writter: writter_csv, single_file = True
[########################################] | 100% Completed |  8min 35.9s
8min 54s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
[########################################] | 100% Completed |  8min 35.9s
8min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_parquet, single_file = False
8min 47s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
compute: threads, file_type: multi-c

In [5]:
get_checkouts("parquet").head()

,BibNumber,ItemBarcode,ItemType,Collection,CallNumber,CheckoutDateTime
0,1842225,0010035249209,acbk,namys,MYSTERY ELKINS1999,05/23/2005 03:20:00 PM
1,1928264,0010037335444,jcbk,ncpic,E TABACK,12/14/2005 05:56:00 PM
2,1982511,0010039952527,jcvhs,ncvidnf,VHS J796.2 KNOW_YO 2000,08/11/2005 01:52:00 PM
3,2026467,0010040985615,accd,nacd,CD 782.421642 Y71T,10/19/2005 07:47:00 PM
4,2174698,0010047696215,jcbk,ncpic,E KROSOCZ,12/29/2005 03:42:00 PM


In [8]:
get_inventory("parquet").head()

,BibNum,Title,Author,ISBN,PublicationYear,Publisher,Subjects,ItemType,ItemCollection,FloatingItem,ItemLocation,ItemCount
ReportDate,,,,,,,,,,,,
09/01/2017,3011076,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie","1481425730, 1481425749, 9781481425735, 9781481...",2014.,"Simon Spotlight,","Musicians Fiction, Bullfighters Fiction, Best ...",jcbk,ncrdr,Floating,qna,1
09/01/2017,2248846,"Naruto. Vol. 1, Uzumaki Naruto / story and art...","Kishimoto, Masashi, 1974-",1569319006,"2003, c1999.","Viz,","Ninja Japan Comic books strips etc, Comic book...",acbk,nycomic,None,lcy,1
09/01/2017,3209270,"Peace, love & Wi-Fi : a ZITS treasury / by Jer...","Scott, Jerry, 1955-","144945867X, 9781449458676",2014.,"Andrews McMeel Publishing,",Duncan Jeremy Fictitious character Comic books...,acbk,nycomic,None,bea,1
09/01/2017,1907265,The Paris pilgrims : a novel / Clancy Carlile.,"Carlile, Clancy, 1930-",0786706155,c1999.,"Carroll & Graf,","Hemingway Ernest 1899 1961 Fiction, Biographic...",acbk,cafic,None,cen,1
09/01/2017,1644616,"Erotic by nature : a celebration of life, of l...",None,094020813X,"1991, c1988.","Red Alder Books/Down There Press,","Erotic literature American, American literatur...",acbk,canf,None,cen,1
